# Python Tutorial: Level 5

### For the OVGU Cognitive Neuroscience Master's course H.3: Projektseminar

### Taught by Reshanne Ruhnau

I would suggest following this course with the [Spyder environment](https://www.spyder-ide.org/) because it allows you to visualize your work nicely (and it has a familiar design if you are coming from MATLAB).

### <a id='toc'>Table of Contents</a>

[Level 5: PsychoPy - Clocks and timing](#level5)

- [Wait](#wait)
- [Clocks](#clock)
- [Frame-based timing](#frame)

[Level 5 Exercises](level5_exercises.ipynb)

BACK TO [Level 0: A New Language](python_tutorial.ipynb)
<br />BACK TO [Level 1: Manipulating Variables](level1.ipynb)
<br />BACK TO [Level 2: Conditionals and Loops](level2.ipynb)
<br />BACK TO [Level 3: PsychoPy 101](level3.ipynb)
<br />BACK TO [Level 4: PsychoPy - Showing windows and stimuli](level4.ipynb)

<br />ONWARD TO [Level 6: PsychoPy - Response collection and save data](level6.ipynb)

## <a id='level5'>Level 5: PsychoPy - Clocks and timing</a>

So far we have been controlling window flips using keypresses, but in a real experiment you will usually want window flips to occur automatically following some kind of timer (draw stimulus, flip window, wait 2 seconds. draw stimulus, flip window, etc.). To do this in psychopy, we need the "core" module:

In [ ]:
from psychopy import core

Then if you type "core." followed by the "tab" key, this will bring up a list of functions you can use with this module. The simplest way to make a window wait until the next flip is to use <a id='wait'>core.wait</a>. This is the easiest but least flexible / least accurate timing function.

All you have to do is replace "event.waitKeys()" in the trial loop with "core.wait()", and the wait time in seconds within the parentheses:

In [ ]:
#So this:
        #-draw stimulus
        my_image.draw() #draw
        win.flip() #show
        event.waitKeys() #wait for keypress

#Becomes:
        #-draw stimulus
        my_image.draw() #draw
        win.flip() #show
        core.wait(2) #wait for 2 seconds

You need to define the wait time for every stimulus that will be shown on a trial. This includes an initial fixation, the image, and then the "end of trial" message. You can add these to your experiment script in the exercises.

[TO THE WAIT EXERCISES](#level5_exercises.ipynb#wait_ex)

[BACK TO TABLE OF CONTENTS](#toc)

"wait" is easy to implement, but you it is imprecise and will also run into problems if you want to present dynamic or changing stimuli. "wait" will essentially freeze the window, so more complex stimuli are not possible with this function. A more precise and flexible option is to use a <a id='clock'>Clock</a>. Clock is psychopy's internal experiment timer. As soon as you define a clock, it starts counting up until the experiment ends or the clock is refreshed. For example:

In [ ]:
my_timer = core.Clock() #define a clock
my_timer.getTime() #get time on the clock

It is important to know that re-defining or resetting my_timer inside or outside of a loop will change the timing that is counted.

If you define my_timer at the beginning of the trial loop, and print "getTime" at the end of the trial loop, you will get the total trial time. This will reset for every trial because you refresh my_timer by redefining it within the loop.

If you define my_timer within the block loop, and print "getTime" at the end of the block loop, you will get the total block time.

If you want to know both the total trial time and the total block time, you can define several clocks outside of either loop, and simply refresh each clock within each loop, respectively:

In [ ]:
block_timer = core.Clock()
trial_timer = core.Clock()

for block in range(nBlocks):
    block_timer.reset()
    
    for trial in range(nTrials):
        trial_timer.reset()
        #run the experiment...
        
        trial_timer.getTime() #get time at the end of the trial
        
    block_timer.getTime() #get block time (remember to indent properly)
    
win.close()       

Clocks are also very handy in defining stimulus timing, and are more accurate than core.wait. To do this, you need to wrap your stimulus in a while loop, and tell your experiment "while the clock is between these 2 times, present the stimulus". This loop is nested inside your trial loop, so instead of this:

In [ ]:
for block in range(nBlocks):    
    #=====================
    #TRIAL SEQUENCE
    #=====================    
    for trial in range(nTrials):
        #-set stimuli and stimulus properties for the current trial
        my_image.image = os.path.join(image_dir,stims[trial]) #point to a different filename for each image
        #-empty keypresses
        
        #=====================
        #START TRIAL
        #=====================   
        #-draw stimulus
        my_image.draw() #draw
        win.flip() #show
        event.waitKeys() #wait for keypress
        
win.close()           

You have this:

In [ ]:
pres_timer = core.Clock() #define the clock at the beginning of the experiment

for block in range(nBlocks):
    #=====================
    #TRIAL SEQUENCE
    #=====================    
    for trial in range(nTrials):
        #-set stimuli and stimulus properties for the current trial
        my_image.image = os.path.join(image_dir,stims[trial]) #point to a different filename for each image
        #-empty keypresses
        
        #=====================
        #START TRIAL
        #=====================   
        pres_timer.reset() #reset stimulus presentation timer right before the first stimulus should appear
        #-draw stimulus
        while pres_timer.getTime() <=2: #2 seconds
            my_image.draw() #draw
            win.flip() #show
            
win.close()               

And you can add multiple while loops to present sequential stimuli:

In [ ]:
        pres_timer.reset() #reset stimulus presentation timer right before the first stimulus should appear
        #-draw stimulus
        while pres_timer.getTime() <=1: #1 second
            fix_text.draw() #draw
            win.flip() #show  
        while 1 < pres_timer.getTime() <=3: #2 seconds
            my_image.draw() #draw
            win.flip() #show  

To present simultaneous stimuli (for example, a constant fixation cross), you have to draw it again within the next while loop:

In [ ]:
        pres_timer.reset() #reset stimulus presentation timer right before the first stimulus should appear
        #-draw stimulus
        while pres_timer.getTime() <=1: #1 second
            fix_text.draw() #draw
            win.flip() #show  
        while 1 < pres_timer.getTime() <=3: #2 seconds
            my_image.draw() #draw
            fix_text.draw() #draw fixation on top of the image
            win.flip() #show  

You can also use a CountdownTimer, which is similar to Clock, except instead of counting up, it counts - you guessed it - down. In that case you simply have to add time to your CountdownTimer after resetting the clock, and invert the times defined in your while loops:

In [ ]:
pres_timer = core.CountdownTimer() #define the countdown clock at the beginning of the experiment

for block in range(nBlocks):
    #=====================
    #TRIAL SEQUENCE
    #=====================    
    for trial in range(nTrials):
        #-set stimuli and stimulus properties for the current trial
        my_image.image = os.path.join(image_dir,stims[trial]) #point to a different filename for each image
        #-empty keypresses
        
        #=====================
        #START TRIAL
        #=====================   
        pres_timer.reset() #reset stimulus presentation timer right before the first stimulus should appear
        pres_timer.add(2) #add 2 seconds because your trial is 2 seconds
        #-draw stimulus
        while pres_timer.getTime() >0: #2 seconds
            my_image.draw() #draw
            win.flip() #show
            
win.close()               

And you can add stimuli (and trial time) as necessary:

In [ ]:
        pres_timer.reset() #reset stimulus presentation timer right before the first stimulus should appear
        pres_timer.add(3) #add 3 seconds because your trial is 3 seconds
        #-draw stimulus
        while pres_timer.getTime() >= 2: #1 second
            fix_text.draw() #draw
            win.flip() #show  
        while 0 <= pres_timer.getTime() < 2: #2 seconds
            my_image.draw() #draw
            fix_text.draw() #draw fixation on top of the image
            win.flip() #show  

[TO THE CLOCK EXERCISES](#level5_exercises.ipynb#clock_ex)

[BACK TO TABLE OF CONTENTS](#toc)

You can use clocks for most experiment timing needs, but what if you want millisecond precision? As precise as clocks are, they are not perfectly accurate because the duration of your stimuli are limited by your monitor's refresh rate. If your monitor has a refresh frequency of 60 Hz (pretty common), that means your computer screen refreshes 60 times in 1 second = 1/60 = 16.666..ms frames. Therefore, a stimulus can be presented for 16.666 ms, 33.333 ms, 50 ms, 66.666 ms...etc, but nothing in between. For millisecond precision based on your hardware limitations, you should therefore use <a id='frame'>frame-based timing</a>.

The first thing you need to find out is your monitor's refresh rate. Whatever it is (x), define the refresh rate as 1.0/x:

In [1]:
refresh=1.0/60.0 #single frame duration in seconds
print(1/(1/60))

60.0


Then you can define your stimulus durations based on this number:

In [2]:
#So this:
fix_dur = 1.0 #1 sec
image_dur = 2.0 #2 sec
text_dur = 1.5 #1.5 sec

#Becomes:
fix_frames = fix_dur / refresh
image_frames = image_dur / refresh
text_frames = text_dur / refresh

print("Seconds:", fix_dur, image_dur, text_dur)
print("Frames:", fix_frames, image_frames, text_frames)

Seconds: 1.0 2.0 1.5
Frames: 60.0 120.0 90.0


When you are deciding upon stimulus presentation times, it is good if your durations in seconds equate to whole numbers of frames as shown sbove. This will give you the most accurate timing whether you use frame-based or clock-based timing. Here is how you can implement frame-based timing into your trial loop:

In [ ]:
fix_frames = int(fix_dur / refresh) #whole number
image_frames = int(image_dur / refresh) #whole number
text_frames = int(text_dur / refresh) #whole number
total_frames = int(fix_frames + image_frames + text_frames) #the total number of frames to be presented on a trial

for block in range(nBlocks):
    #=====================
    #TRIAL SEQUENCE
    #=====================    
    for trial in range(nTrials):
        #-set stimuli and stimulus properties for the current trial
        my_image.image = os.path.join(image_dir,stims[trial]) #point to a different filename for each image
        #-empty keypresses
        
        #=====================
        #START TRIAL
        #=====================   
        for frameN in range(total_frames): #for the whole trial...
            #-draw stimulus
            if 0 <= frameN <= fix_frames: #number of frames for fixation
                fix.draw() #draw
                win.flip() #show
            if fix_frames < frameN <= (fix_frames+image_frames): #number of frames for image after fixation
                fix.draw() #draw
                win.flip() #show    
            if (fix_frames+image_frames) < frameN < (fix_frames+image_frames+text_frames): #number of frames for the final text stimulus
                fix.draw() #draw
                win.flip() #show   
                
win.close()                

This is the most accurate way to measure timing... IF you are not dropping frames. Your experiment may drop frames for varius reasons - if you are running simultaneous programs, if you are running your experiment from a virtual environment, etc. This can lead to lags on the order of several milliseconds. If there is a lag, the experiment will wait until another frame is completed, thus "dropping" the last frame. You can check if your experiment is dropping frames with the following code:

In [ ]:
from psychopy import logging #this can output various information about your experiment

win.recordFrameIntervals = True #record frames
win.refreshThreshold = 1.0/60.0 + 0.004 #give the monitor refresh rate plus a few ms tolerance (usually 4ms)

# Set the log module to report warnings to the standard output window (default is errors only).
logging.console.setLevel(logging.WARNING)

#-------------------  
#EXPERIMENT CODE HERE
#for block in range(nBlocks):
    #=====================
    #TRIAL SEQUENCE
    #=====================    
    #define stimulus params
        #=====================
        #START TRIAL
        #=====================   
        #run trial
#-------------------           

        #this will print total number of frames dropped following every trial
        print('Overall, %i frames were dropped.' % win.nDroppedFrames) #print number of dropped frames

win.close()

If your experiment keeps dropping frames, you may actually notice a lag in your stimulus presentation when using frame-based timing. In this case, you can try to make your experiment run more efficiently (closing multiple programs, running the experiment outside of a virtual environment), but if this is not possible, it is better to use clock-based timing.

[TO THE FRAME-BASED TIMING EXERCISES](#level5_exercises.ipynb#frame_ex)

[BACK TO TABLE OF CONTENTS](#toc)